In [1]:
# In MSR
# survey_collection_name = 'internal_pilot_collection'
# user_collection_name = 'internal_user_collection'

# To doctors pilot
# survey_collection_name = 'doc_pilot_survey_collection'
# user_collection_name = 'doc_pilot_user_collection'

# Phase 1 Section 1
# survey_collection_name = 'p1_s1_survey_collection'
# user_collection_name = 'p1_s1_user_collection'

import uuid
import numpy as np
import pandas as pd
import asyncio
import httpx
from tqdm.notebook import tqdm
from typing import Dict, List
from cataract_doc_study.dependency_setup import user_client, survey_client
user_client = httpx.AsyncClient()
client = httpx.AsyncClient()
class Node:
    def __init__(self, value):
        self.id = str(uuid.uuid4())
        self.action_type = value["action_type"]
        self.update_info = value.get("update_info", "")
        self.timestamp = value["timestamp"]
        self.visited = 1
        self.parent = None
        self.children = []
        self.outgoing_edge_count: Dict[Node, int] = dict()
        self.active_child = None
        self.final_node = False

/home/rash598/doctor_sys_3/cataract-doc-study/.venv/lib/python3.10/site-packages/motor/core.py:171: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  delegate = self.__delegate_class__(*args, **kwargs)


In [2]:
docs_df = pd.read_json("/home/rash598/doctor_sys_3/cataract-doc-study/cataract_doc_study/doctors_and_questions/docs.json")
questions_df = pd.read_json("/home/rash598/doctor_sys_3/cataract-doc-study/cataract_doc_study/doctors_and_questions/questions.json")

In [3]:
async def get_doc_question_set(doc_id):
    url = "https://cataract-doctor-study-dzb2hfc5h4aqbafk.eastus-01.azurewebsites.net/get_user"
    params = {"user_id": doc_id}

    response = await user_client.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    return None

async def get_question_data(doctor_id, question_id, condition_id):
    url = "https://cataract-doctor-study-dzb2hfc5h4aqbafk.eastus-01.azurewebsites.net/get_answer"
    params = {
        "user_id": doctor_id,
        "question_id": question_id,
        "condition_id": condition_id
    }
    response = await client.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    return None

In [4]:
def levenshtein_distance_operations(sent1, sent2):
    # Tokenize sentences into words
    s1 = sent1.strip().split()
    s2 = sent2.strip().split()
    
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    ops = [[(0, 0, 0)] * (n + 1) for _ in range(m + 1)]  # (insert, delete, replace)

    for i in range(m + 1):
        dp[i][0] = i
        ops[i][0] = (0, i, 0)

    for j in range(n + 1):
        dp[0][j] = j
        ops[0][j] = (j, 0, 0)

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
                ops[i][j] = ops[i - 1][j - 1]
            else:
                insert = dp[i][j - 1] + 1
                delete = dp[i - 1][j] + 1
                replace = dp[i - 1][j - 1] + 1

                if insert <= delete and insert <= replace:
                    dp[i][j] = insert
                    ins, dele, rep = ops[i][j - 1]
                    ops[i][j] = (ins + 1, dele, rep)
                elif delete <= insert and delete <= replace:
                    dp[i][j] = delete
                    ins, dele, rep = ops[i - 1][j]
                    ops[i][j] = (ins, dele + 1, rep)
                else:
                    dp[i][j] = replace
                    ins, dele, rep = ops[i - 1][j - 1]
                    ops[i][j] = (ins, dele, rep + 1)

    insertions, deletions, substitutions = ops[m][n]
    return dp[m][n], insertions, deletions, substitutions


In [5]:
def create_graph(activity_tracker):
    nodes_dict: Dict[str, Node] = {}
    start_node = None
    prev_node = None
    curr_node = None
    for action in activity_tracker:
        if action["action_type"] == "question_start":
            curr_node = Node(action)
            nodes_dict[curr_node.id] = curr_node
            start_node = curr_node
            prev_node = curr_node
        elif action["action_type"] == "update_answer":
            curr_node = Node(action)
            nodes_dict[curr_node.id] = curr_node
            prev_node.children.append(curr_node)
            if curr_node.id not in prev_node.outgoing_edge_count:
                prev_node.outgoing_edge_count[curr_node.id] = 0
            prev_node.active_child = curr_node
            curr_node.parent = prev_node
            prev_node = curr_node
        elif action["action_type"] == "go_left":
            temp_node = prev_node
            prev_node = prev_node.parent
            prev_node.visited += 1
            if prev_node.id not in temp_node.outgoing_edge_count:
                temp_node.outgoing_edge_count[prev_node.id] = 0
            temp_node.outgoing_edge_count[prev_node.id] += 1
        elif action["action_type"] == "go_right":
            temp_node = prev_node
            prev_node = prev_node.active_child
            prev_node.visited += 1
            if prev_node.id not in temp_node.outgoing_edge_count:
                temp_node.outgoing_edge_count[prev_node.id] = 0
            temp_node.outgoing_edge_count[prev_node.id] += 1
        elif action["action_type"] == "submit_answer":
            prev_node.final_node = True
    return start_node, nodes_dict

def final_answer_not_last_node(nodes_dict):
    for node in nodes_dict.values():
        if node.final_node and node.active_child is not None:
            return True
    return False

def ideal_ops_to_submit(node):
    # 1 is for submit action
    len = 1
    while not node.final_node:
        len += 1
        node = node.active_child
    return len

def number_of_branches(nodes_dict):
    branches = 0
    for node in nodes_dict.values():
        if node.active_child is None:
            branches += 1
    return branches

def number_of_ops_to_submit(activity_tracker):
    ops = 0
    for action in activity_tracker:
        if action["action_type"] == "update_answer" or action["action_type"] == "submit_answer" or action["action_type"] == "go_left" or action["action_type"] == "go_right":
            ops += 1
    return ops
def extract_info_from_activity_tracker(activity_tracker):
    # print(activity_tracker)
    start_node, nodes_dict = create_graph(activity_tracker)
    num_nodes = len(nodes_dict)
    is_previous_answer_selected = final_answer_not_last_node(nodes_dict)
    ideal_ops = ideal_ops_to_submit(start_node)
    overall_ops = number_of_ops_to_submit(activity_tracker)
    num_branches = number_of_branches(nodes_dict)
    cumulative_llm_time = 0
    num_comments = 0
    comments_length = []
    comments = []
    unused_nodes = num_nodes - (ideal_ops - 1)  # -1 for the submit action
    penatly = num_nodes + 2*unused_nodes + 3*num_branches
    for activity in activity_tracker:
        action_type = activity["action_type"]
        if action_type == "update_answer":
            end_time = activity["llm_end_timestamp"]
            start_time = activity["llm_start_timestamp"]
            update_info = activity["update_info"].split(" ")
            duration = end_time - start_time
            cumulative_llm_time += duration
            num_comments += 1
            comments_length.append(len(update_info))
            comments.append(activity["update_info"])
    return num_comments, comments_length, comments, cumulative_llm_time, is_previous_answer_selected, ideal_ops, overall_ops, num_branches, penatly

async def process_question_list(question_list, doctor_id, doctor_email, response_dump_df):
    question_data_list = []
    tasks = []
    for question in question_list:
        question_id = question["question_id"]
        condition_id = question["condition_id"]
        get_data_task = get_question_data(doctor_id, question_id, condition_id)
        tasks.append(get_data_task)
    question_data_list = await asyncio.gather(*tasks)

    for i, question in enumerate(question_list):
        # print(question)
        question_id = question["question_id"]
        condition_id = question["condition_id"]
        question_text = questions_df.loc[questions_df["id"] == question_id, "question"].values[0]
        if condition_id == 0:
            original_answer = ""
        else:
            original_answer = questions_df.loc[questions_df["id"] == question_id, "answer"].values[0]
        questiond_data = question_data_list[i]
        if questiond_data is None:
            continue
        # print(questiond_data)
        final_answer = questiond_data["question_metadata"]["final_answer"]
        duration = questiond_data["question_metadata"]["duration"]
        edit_distance, insertions, deletions, substitutions = levenshtein_distance_operations(original_answer, final_answer)
        if condition_id == 2:
            num_comments, comments_length, comments, cumulative_llm_time, is_previous_answer_selected, ideal_ops, overall_ops, num_branches, penalty  = extract_info_from_activity_tracker(questiond_data["activity_tracker"])
        else:
            num_comments = 0
            comments_length = []
            comments =[]
            cumulative_llm_time = 0
            is_previous_answer_selected = False
            ideal_ops = 1
            overall_ops = 1
            num_branches = 1
            penalty = 0

        question_data = {
            "doctor_id": doctor_id,
            "doctor_email": doctor_email,
            "question_id": question_id,
            "condition_id": condition_id,
            "original_answer": original_answer,
            "final_answer": final_answer,
            "duration": duration,
            "edit_distance": edit_distance,
            "insert": insertions,
            "delete": deletions,
            "substitute": substitutions,
            "num_comments": num_comments,
            "comments_length": comments_length,
            "comments": comments,
            "cumulative_llm_time": cumulative_llm_time,
            "is_previous_answer_selected": is_previous_answer_selected,
            "total_branches": num_branches,
            "number_of_ops_to_submit": overall_ops,
            "ideal_ops_to_submit": ideal_ops,
            "penalty": penalty
        }
        response_dump_df = pd.concat([response_dump_df, pd.DataFrame([question_data])], ignore_index=True)
    return response_dump_df


In [9]:
response_dump_df = pd.DataFrame(columns=["doctor_id", "doctor_email", "condition_id", "question_id", "original_answer", "final_answer", "duration", "edit_distance", "insert", "delete", "substitute", "num_comments", "comments_length", "comments", "cumulative_llm_time", "is_previous_answer_selected", "total_branches", "number_of_ops_to_submit", "ideal_ops_to_submit", "penalty"])
for index, row in tqdm(docs_df.iterrows(), total=len(docs_df), desc="Fetching question sets"):
    doctor_id = row["id"]
    doctor_email = row["email"]
    question_set = await get_doc_question_set(doctor_id)
    question_list = question_set["questions_list"]
    response_dump_df = await process_question_list(question_list, doctor_id, doctor_email, response_dump_df)

Fetching question sets:   0%|          | 0/12 [00:00<?, ?it/s]

/tmp/ipykernel_114423/2077210486.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  response_dump_df = pd.concat([response_dump_df, pd.DataFrame([question_data])], ignore_index=True)


In [10]:
response_dump_df['duration'] = (response_dump_df['duration'] / 1000).round(3)  # Convert milliseconds to seconds
response_dump_df['cumulative_llm_time'] = (response_dump_df['cumulative_llm_time'] / 1000).round(3)  # Convert milliseconds to seconds

In [11]:
# response_dump_df.to_excel("doc_pilot_survey_response_dump.xlsx", index=False)
response_dump_df

,doctor_id,doctor_email,condition_id,question_id,original_answer,final_answer,duration,edit_distance,insert,delete,substitute,num_comments,comments_length,comments,cumulative_llm_time,is_previous_answer_selected,total_branches,number_of_ops_to_submit,ideal_ops_to_submit,penalty
0,Moh,mohit@gmail.com​,1,Q9,"Yes, post-surgery eye drops should be applied ...","Yes, post-surgery eye drops should be applied ...",5.0,0,0,0,0,0,[],[],0.0,False,1,1,1,0
1,Rah,rahul@gmail.com​,0,Q5,,xx,3.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
2,Rah,rahul@gmail.com​,0,Q6,,xx,1.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
3,Rah,rahul@gmail.com​,0,Q7,,xx,1.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
4,Rah,rahul@gmail.com​,0,Q8,,xx,1.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
5,Rah,rahul@gmail.com​,0,Q69,,xx,1.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
6,Rah,rahul@gmail.com​,0,Q70,,xx,4.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
7,Rah,rahul@gmail.com​,0,Q71,,xx,1.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
8,Rah,rahul@gmail.com​,0,Q72,,xx,1.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0
9,Rah,rahul@gmail.com​,0,Q73,,xx,1.0,1,1,0,0,0,[],[],0.0,False,1,1,1,0


In [7]:
def get_activity_trackers(question_list, doctor_id, doctor_email, activity_trackers: list):
    
    for question in question_list:
        # print(question)
        question_id = question["question_id"]
        condition_id = question["condition_id"]
        question_text = questions_df.loc[questions_df["id"] == question_id, "question"].values[0]
        if condition_id != 2:
            continue
        questiond_data = get_question_data(doctor_id, question_id, condition_id)
        if questiond_data is None:
            continue
        activity_trackers.append(questiond_data["activity_tracker"])

activity_trackers = []
for index, row in tqdm(docs_df.iterrows(), total=len(docs_df), desc="Fetching question sets"):
    doctor_id = row["id"]
    doctor_email = row["email"]
    question_set = get_doc_question_set(doctor_id)
    question_list = question_set["questions_list"]
    get_activity_trackers(question_list, doctor_id, doctor_email, activity_trackers)

Fetching question sets:   0%|          | 0/5 [00:00<?, ?it/s]

In [19]:
class Node:
    def __init__(self, value):
        self.id = str(uuid.uuid4())
        self.action_type = value["action_type"]
        self.update_info = value.get("update_info", "")
        self.timestamp = value["timestamp"]
        self.visited = 1
        self.parent = None
        self.children = []
        self.outgoing_edge_count: Dict[Node, int] = dict()
        self.active_child = None
        self.final_node = False

In [28]:
def create_graph(activity_tracker):
    nodes_dict: Dict[str, Node] = {}
    start_node = None
    prev_node = None
    curr_node = None
    activity_tracker = activity_trackers[0]
    for action in activity_tracker:
        if action["action_type"] == "question_start":
            curr_node = Node(action)
            nodes_dict[curr_node.id] = curr_node
            start_node = curr_node
            prev_node = curr_node
        elif action["action_type"] == "update_answer":
            curr_node = Node(action)
            nodes_dict[curr_node.id] = curr_node
            prev_node.children.append(curr_node)
            if curr_node.id not in prev_node.outgoing_edge_count:
                prev_node.outgoing_edge_count[curr_node.id] = 0
            prev_node.active_child = curr_node
            curr_node.parent = prev_node
            prev_node = curr_node
        elif action["action_type"] == "go_left":
            temp_node = prev_node
            prev_node = prev_node.parent
            prev_node.visited += 1
            if prev_node.id not in temp_node.outgoing_edge_count:
                temp_node.outgoing_edge_count[prev_node.id] = 0
            temp_node.outgoing_edge_count[prev_node.id] += 1
        elif action["action_type"] == "go_right":
            temp_node = prev_node
            prev_node = prev_node.active_child
            prev_node.visited += 1
            if prev_node.id not in temp_node.outgoing_edge_count:
                temp_node.outgoing_edge_count[prev_node.id] = 0
            temp_node.outgoing_edge_count[prev_node.id] += 1
        elif action["action_type"] == "submit_answer":
            prev_node.final_node = True
    return start_node, nodes_dict

def traverse_graph(node):
    while node is not None:
        print(f"Node_id: {node.id}, Action: {node.action_type}, Update Info: {node.update_info}, Timestamp: {node.timestamp}, Visited: {node.visited}, Final Node: {node.final_node}, outgoing edges: {node.outgoing_edge_count}")
        node = node.active_child

In [29]:
activity_tracker = activity_trackers[0]
start_node, nodes_dict = create_graph(activity_tracker)
traverse_graph(start_node)

Node_id: 8024cd73-1874-4898-8dd0-de815d23d25e, Action: question_start, Update Info: None, Timestamp: 1748428997819, Visited: 3, Final Node: False, outgoing edges: {'e368f679-966e-4a56-a983-dbdfb74f203d': 2}
Node_id: e368f679-966e-4a56-a983-dbdfb74f203d, Action: update_answer, Update Info: remove the last setence. 
remove the first sentence.
remove the second last sentence., Timestamp: 1748429029100, Visited: 6, Final Node: False, outgoing edges: {'8024cd73-1874-4898-8dd0-de815d23d25e': 2, 'a818c3d3-1d71-43e5-afe2-3fc570e8569d': 3}
Node_id: a818c3d3-1d71-43e5-afe2-3fc570e8569d, Action: update_answer, Update Info: revert back, Timestamp: 1748429085955, Visited: 7, Final Node: True, outgoing edges: {'e368f679-966e-4a56-a983-dbdfb74f203d': 3, '290c6222-0967-4625-a001-144d9c9b38b2': 2}
Node_id: 290c6222-0967-4625-a001-144d9c9b38b2, Action: update_answer, Update Info: go back, Timestamp: 1748429102478, Visited: 6, Final Node: False, outgoing edges: {'9fee8b8b-74a9-464f-8a26-2cf4a149c7fe': 2,

In [36]:
import json
for node in nodes_dict.values():
    print(node.__dict__)

{'id': '8024cd73-1874-4898-8dd0-de815d23d25e', 'action_type': 'question_start', 'update_info': None, 'timestamp': 1748428997819, 'visited': 3, 'parent': None, 'children': [<__main__.Node object at 0x7de3a4d7ac50>], 'outgoing_edge_count': {'e368f679-966e-4a56-a983-dbdfb74f203d': 2}, 'active_child': <__main__.Node object at 0x7de3a4d7ac50>, 'final_node': False}
{'id': 'e368f679-966e-4a56-a983-dbdfb74f203d', 'action_type': 'update_answer', 'update_info': 'remove the last setence. \nremove the first sentence.\nremove the second last sentence.', 'timestamp': 1748429029100, 'visited': 6, 'parent': <__main__.Node object at 0x7de3a4d7b340>, 'children': [<__main__.Node object at 0x7de3a4d7a560>], 'outgoing_edge_count': {'8024cd73-1874-4898-8dd0-de815d23d25e': 2, 'a818c3d3-1d71-43e5-afe2-3fc570e8569d': 3}, 'active_child': <__main__.Node object at 0x7de3a4d7a560>, 'final_node': False}
{'id': 'a818c3d3-1d71-43e5-afe2-3fc570e8569d', 'action_type': 'update_answer', 'update_info': 'revert back', 'tim

In [54]:
import networkx as nx
from pyvis.network import Network
def build_graph_from_nodes_dict(nodes_dict: Dict[str, Node]) -> nx.DiGraph:
    G = nx.DiGraph()

    for node_id, node in nodes_dict.items():
        # Add the node
        G.add_node(node_id,
                   action_type=node.action_type,
                   update_info=node.update_info,
                   timestamp=node.timestamp,
                   visited=node.visited,
                   final_node=node.final_node)

        # Add edges (target_node is a string ID)
        for target_node_id, count in node.outgoing_edge_count.items():
            G.add_edge(node_id, target_node_id, weight=count)

    return G

def visualize_graph_pyvis(G: nx.DiGraph, filename="mindmap.html"):
    net = Network(height="750px", width="100%", directed=True)

    for node_id, data in G.nodes(data=True):
        label = f"{data['action_type']}\n{data['update_info']}"
        net.add_node(node_id, label=label, title=f"Visited: {data['visited']}")

    for src, tgt, edge_data in G.edges(data=True):
        net.add_edge(src, tgt, value=edge_data['weight'], title=f"Weight: {edge_data['weight']}")

    net.show(filename)

In [101]:
from dash import Dash, html, Output, Input, State
import dash_cytoscape as cyto
from typing import Dict

def visualize_nodes_mindmap(nodes_dict: Dict[str, 'Node'], port: int = 8050):
    elements = []
    root_ids = []

    full_node_data = {}

    for node_id, node in nodes_dict.items():
        label = f"{node.action_type}\n{node.update_info}" if node.update_info else node.action_type

        if node.final_node:
            color = "#4CAF50"  # Green
        elif node.action_type == "question_start":
            color = "#add8e6"  # Light blue
            root_ids.append(node_id)
        else:
            color = "#2b7ce9"

        elements.append({
            'data': {'id': node_id, 'label': label, 'color': color}
        })

        full_node_data[node_id] = {
            'id': node.id,
            'action_type': node.action_type,
            'update_info': node.update_info,
            'timestamp': node.timestamp,
            'visited': node.visited,
            'final_node': node.final_node
        }

        for target_id, count in node.outgoing_edge_count.items():
            elements.append({
                'data': {
                    'source': node_id,
                    'target': target_id,
                    'label': f'×{count}'
                }
            })

    return elements, root_ids, full_node_data
    


In [102]:
elements, root_ids, full_node_data = visualize_nodes_mindmap(nodes_dict)
app = Dash(__name__)
app.title = "Mind Map with Popup Info"

app.layout = html.Div([
    html.H1("Mind Map Tree (Click Node for Info)", style={"textAlign": "center"}),
    html.Div(
        [
            cyto.Cytoscape(
                id='cytoscape-tree',
                elements=elements,
                style={'width': '100%', 'height': '600px', 'position': 'relative'},
                layout={
                    'name': 'breadthfirst',
                    'roots': root_ids,
                    'directed': True,
                    'spacingFactor': 1.5
                },
                stylesheet=[
                    {
                        'selector': 'node',
                        'style': {
                            'content': 'data(label)',
                            'text-wrap': 'wrap',
                            'text-valign': 'center',
                            'text-halign': 'center',
                            'background-color': 'data(color)',
                            'color': '#000',
                            'font-size': '12px',
                            'width': 'label',
                            'height': 'label',
                            'padding': '10px'
                        }
                    },
                    {
                        'selector': 'edge',
                        'style': {
                            'label': 'data(label)',
                            'curve-style': 'bezier',
                            'target-arrow-shape': 'triangle',
                            'arrow-scale': 1.5,
                            'line-color': '#888',
                            'target-arrow-color': '#888',
                            'font-size': '10px',
                            'color': '#333'
                        }
                    }
                ]
            ),
            # Popup container, initially hidden
            html.Div(
                id='node-popup',
                style={
                    'position': 'absolute',
                    'backgroundColor': 'white',
                    'border': '1px solid #ccc',
                    'padding': '10px',
                    'border-radius': '5px',
                    'box-shadow': '2px 2px 6px rgba(0,0,0,0.2)',
                    'display': 'none',
                    'zIndex': 9999,
                    'maxWidth': '250px',
                    'pointerEvents': 'auto',
                }
            )
        ],
        style={'position': 'relative', 'width': '100%', 'height': '600px'}
    )
])

@app.callback(
    Output('node-popup', 'style'),
    Output('node-popup', 'children'),
    Input('cytoscape-tree', 'tapNodeData'),
    Input('cytoscape-tree', 'tapNode'),
    prevent_initial_call=True
)
def display_popup(node_data, node_event):
    if not node_data or not node_event:
        # Hide popup if no node clicked
        return {'display': 'none'}, ""

    node_id = node_data['id']
    info = full_node_data.get(node_id, {})

    # Get clicked node position in pixels relative to Cytoscape container
    pos = node_event['position']  # dict with 'x' and 'y' relative to Cytoscape coordinate system

    # Approximate conversion for screen position (can tweak offsets if needed)
    left = pos['x'] + 20
    top = pos['y'] + 20

    style = {
        'position': 'absolute',
        'left': f'{left}px',
        'top': f'{top}px',
        'backgroundColor': 'white',
        'border': '1px solid #ccc',
        'padding': '10px',
        'border-radius': '5px',
        'box-shadow': '2px 2px 6px rgba(0,0,0,0.2)',
        'display': 'block',
        'zIndex': 9999,
        'maxWidth': '250px',
        'pointerEvents': 'auto',
    }

    content = html.Div([
        html.Strong(f"Node ID: {node_id}"),
        html.Br(),
        f"Action Type: {info.get('action_type')}",
        html.Br(),
        f"Update Info: {info.get('update_info')}",
        html.Br(),
        f"Timestamp: {info.get('timestamp')}",
        html.Br(),
        f"Visited: {info.get('visited')}",
        html.Br(),
        f"Final Node: {info.get('final_node')}",
    ])

    return style, content

def run_dash_app(app, port):
    app.run(debug=False, port=port, use_reloader=False)
    return f"http://127.0.0.1:{port}"
run_dash_app(app, port=8050)

'http://127.0.0.1:8050'